In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import glob
import fiona
import rasterio
import numpy as np
import rasterio.mask
import tensorflow as tf
import tensorflow_addons as tfa

from models import loss
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from models.metrics import iou, dice_coef
from models.callback.save_best import SavebestweightsandEarlyStopping

import warnings
warnings.filterwarnings("ignore")

In [3]:
def conv_block(input_feature, filters, kernel_size=(1,1), strides=1, padding='same', use_relu=False):
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding,
                                        kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(input_feature)
    x = tf.keras.layers.BatchNormalization()(x)
    if use_relu:
        x = tf.keras.layers.Activation('relu')(x)
    return x

def depthwise_conv_block(input_feature, kernel_size, strides=1, padding='same', r=1):
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=kernel_size, strides=strides, padding=padding, 
                                        dilation_rate=r, depthwise_initializer='he_normal',
                                        use_bias=True, bias_initializer='zeros')(input_feature)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def channel_attention_block(input_feature, ratio=8):
    channel = input_feature.shape[-1]

    shared_layer_one = tf.keras.layers.Dense(channel//ratio, activation='relu', kernel_initializer='he_normal',
                                            use_bias=True, bias_initializer='zeros')
    shared_layer_two = tf.keras.layers.Dense(channel, activation='relu', kernel_initializer='he_normal',
                                            use_bias=True, bias_initializer='zeros')
    
    avg_pool = tf.keras.layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = tf.keras.layers.Reshape((1,1,channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input_feature)
    max_pool = tf.keras.layers.Reshape((1,1,channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = tf.keras.layers.Add()([avg_pool,max_pool])
    cbam_feature = tf.keras.layers.Activation('sigmoid')(cbam_feature)
    return tf.keras.layers.multiply([input_feature, cbam_feature])

def spatial_attention_block(input_feature):
    kernel_size = 7
    channel = input_feature.shape[-1]
    cbam_feature = input_feature

    avg_pool = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    max_pool = tf.keras.layers.Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    concat = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])

    cbam_feature = tf.keras.layers.Conv2D(filters = 1, kernel_size=kernel_size, strides=1, padding='same',
                    activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)

    return tf.keras.layers.multiply([input_feature, cbam_feature])

def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """

    cbam_feature = channel_attention_block(cbam_feature, ratio)
    cbam_feature = spatial_attention_block(cbam_feature)
    return cbam_feature

def ADM_block(input_feature):
    filters = input_feature.shape[-1]
    x = conv_block(input_feature, filters=filters, kernel_size=(1,1), strides=1, padding='same', use_relu=True)
    x = depthwise_conv_block(x, kernel_size=(3,3), strides=2, padding='same', r=1)
    x = conv_block(x, filters=filters, kernel_size=(1,1), strides=1, padding='same', use_relu=False)
    x_pool = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2)(input_feature)
    x_concat = tf.keras.layers.concatenate([x, x_pool], axis=-1)
    x = channel_attention_block(x_concat)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def SCBAM_block(input_feature, r=1):
    filters = input_feature.shape[-1] // 4
    x = conv_block(input_feature, filters=filters, kernel_size=(1,1), strides=1, padding='same', use_relu=True)
    x = depthwise_conv_block(x, kernel_size=(3,3), strides=1, padding='same', r=r)
    x = conv_block(x, filters=filters*4, kernel_size=(1,1), strides=1, padding='same', use_relu=False)
    x = channel_attention_block(x)
    x_add = tf.keras.layers.Add()([x, input_feature])
    x = tf.keras.layers.Activation('relu')(x_add)
    return x

def stack_ADM_SCBAM(input_feature):
    x = ADM_block(input_feature)
    x = SCBAM_block(x)
    x = SCBAM_block(x)
    return x

def HDC_block(input_feature, rate_list):
    x = input_feature
    for i in rate_list:
        x = SCBAM_block(x, r=i)
    return x

def multi_HDC_block(input_feature, stage1=[1,2,5,9,1,2,5,9], stage2=[1,2,5,9], stage3=[1,2], stage4=[1]):
    x1 = HDC_block(input_feature, rate_list=stage1)
    x2 = HDC_block(x1, rate_list=stage2)
    x3 = HDC_block(x2, rate_list=stage3)
    x4 = HDC_block(x3, rate_list=stage4)
    x_concat = tf.keras.layers.concatenate([x1,x2,x3,x4,input_feature])
    return x_concat

def DUC_block(input_feature, out_channles=1, upscale=8):
    out_channles = out_channles * (upscale ** 2)
    x = tf.keras.layers.Conv2D(filters=out_channles, kernel_size=(1,1), strides=1, padding='same', use_bias=False)(input_feature)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    outputs = tf.nn.depth_to_space(x, upscale)
    return outputs

def MHA_Net(input_shape, filters=64, num_ADM_SCBAM=3):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_block(inputs, filters=filters, kernel_size=(3, 3))
    x = conv_block(x, filters=filters*2, kernel_size=(3, 3))

    for _ in range(num_ADM_SCBAM):   
        x = stack_ADM_SCBAM(x)

    x = multi_HDC_block(x)
    x = channel_attention_block(x)
    x = conv_block(x, filters=filters*16, kernel_size=(1,1))
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(1,1), strides=1, padding='same',
                                kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(x)
    x = DUC_block(x)
    x = tf.keras.layers.Activation('sigmoid')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    return model

In [4]:
from preprocess.prepare_dataset import data_gen

out_path = '/home/quyet/DATA_ML/Projects/change_detection/new_video/tarmac/bonus_negative'
overlap_mask = os.path.join(out_path, 'mask_cut_crop')
train_dataset, valid_dataset, _, _ = data_gen(os.path.join(overlap_mask, '*.tif'), img_size=512, 
                                                            batch_size=1, N_CLASSES=1, numband=3, 
                                                            split_ratios=0.8, test_data=False, multi=False)

Training:validation = 2035:509


2022-07-21 17:10:58.265207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4998 MB memory:  -> device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:05:00.0, compute capability: 6.1


In [ ]:
model_name = 'mhanet'
mission = 'change_road'
img_size = 512
num_bands = 3
num_class = 1 
batch_size = 1

def lr_decay(epoch):
    initial_learningrate=3e-4
    if epoch < 1:
        return initial_learningrate
    else:
        return initial_learningrate * 0.9 ** (epoch)

if batch_size >1:
    val_batch_size = int(batch_size/2)
else:
    val_batch_size = batch_size
    
print("Init metric function")
if num_class==1:
    recall = tf.keras.metrics.Recall()
    precision = tf.keras.metrics.Precision()
    model_metrics = [precision,
                     recall, 
                     dice_coef,
                     iou, 
                     # tf.keras.metrics.BinaryAccuracy(threshold=0.5)
                    ]
else:
    recall = tf.keras.metrics.Recall()
    precision = tf.keras.metrics.Precision()
    accuracy = tf.keras.metrics.CategoricalAccuracy()
    model_metrics = [precision, recall, dice_coef, iou, accuracy]

root_dir = os.path.dirname(sys.path[0])
data_path = os.path.join(os.path.join(root_dir, 'data', mission))
checkpoint_filepath = os.path.join(root_dir, 'logs', mission, 'tmp')
log_dir = os.path.join(root_dir, 'logs', mission, 'graph')
weights_path = os.path.join(root_dir, 'weights', '%s'%(model_name), model_name+'_'+mission+'_'+str(img_size)+'_'+str(num_class)+'class.h5')
patience = 10

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only= True, 
                                                                monitor='val_loss', mode='min', save_best_only=True)
model_lrscheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=1)
model_lrreduce_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience, min_lr=1e-7, verbose=1)
model_earlystopping_callback = SavebestweightsandEarlyStopping(patience=patience, weights_path=weights_path)
model_endtrainnan_callback = tf.keras.callbacks.TerminateOnNaN()
model_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
model_callbacks = [model_checkpoint_callback, model_lrscheduler_callback,
                    model_lrreduce_callback, model_earlystopping_callback,
                    model_tensorboard_callback,]

model = MHA_Net((img_size,img_size,num_bands))

loss_func = loss.hybrid_loss

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer = optimizer, loss = loss_func,
             metrics = model_metrics
             )

model.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/mhanet/mhanet_change_road_512_1class_train.h5')
history_train = model.fit(train_dataset, batch_size=batch_size, epochs=100, verbose=1, 
                      callbacks=model_callbacks, validation_data=valid_dataset, 
                      validation_batch_size=val_batch_size, use_multiprocessing=True)

Init metric function

Epoch 00001: LearningRateScheduler setting learning rate to 0.0003.
Epoch 1/100


2022-07-21 17:11:38.667683: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-07-21 17:11:38.982682: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-07-21 17:11:40.056078: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 798.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-07-21 17:11:40.076580: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mea

   2035/Unknown - 1718s 825ms/step - loss: 0.3778 - precision: 0.9900 - recall: 0.9449 - dice_coef: 0.9257 - iou: 0.8627
Save best train weights.
Save best val weights.


2022-07-21 17:44:08.433960: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.
2022-07-21 17:44:09.611860: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.
2022-07-21 17:44:10.880899: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1258291200 exceeds 10% of free system memory.


2035/2035 [==============================] - 1988s 958ms/step - loss: 0.3778 - precision: 0.9900 - recall: 0.9449 - dice_coef: 0.9257 - iou: 0.8627 - val_loss: 0.4036 - val_precision: 0.9990 - val_recall: 0.8407 - val_dice_coef: 0.9084 - val_iou: 0.8410 - lr: 3.0000e-04

Epoch 00002: LearningRateScheduler setting learning rate to 0.00027.
Epoch 2/100
 527/2035 [======>.......................] - ETA: 21:08 - loss: 0.3410 - precision: 0.9892 - recall: 0.9557 - dice_coef: 0.9376 - iou: 0.8837

...

 531/2035 [======>.......................] - ETA: 21:05 - loss: 0.3407 - precision: 0.9893 - recall: 0.9554 - dice_coef: 0.9376 - iou: 0.8838

2022-07-21 17:51:40.615650: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 100.03MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


2035/2035 [==============================] - ETA: 0s - loss: 0.3242 - precision: 0.9896 - recall: 0.9643 - dice_coef: 0.9426 - iou: 0.8928
Save best train weights.
Save best val weights.


2022-07-21 18:16:55.347585: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.
2022-07-21 18:16:56.543902: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.


2035/2035 [==============================] - 1967s 966ms/step - loss: 0.3242 - precision: 0.9896 - recall: 0.9643 - dice_coef: 0.9426 - iou: 0.8928 - val_loss: 0.2895 - val_precision: 0.9938 - val_recall: 0.9665 - val_dice_coef: 0.9498 - val_iou: 0.9069 - lr: 2.7000e-04

Epoch 00003: LearningRateScheduler setting learning rate to 0.000243.
Epoch 3/100
 118/2035 [>.............................] - ETA: 27:23 - loss: 0.3114 - precision: 0.9815 - recall: 0.9716 - dice_coef: 0.9458 - iou: 0.8992

In [18]:
# import cv2
# import numpy as np

# from tqdm import tqdm
# from osgeo import gdal
# from postprocess.convert_tif import dilation_obj, remove_small_items, write_image

# def get_im_by_coord(org_im, start_x, start_y,num_band, padding, crop_size, input_size):
#     startx = start_x-padding
#     endx = start_x+crop_size+padding
#     starty = start_y - padding
#     endy = start_y+crop_size+padding
#     result=[]
#     img = org_im[starty:endy, startx:endx]
#     img = img.swapaxes(2,1).swapaxes(1,0)
#     for chan_i in range(num_band):
#         result.append(cv2.resize(img[chan_i],(input_size, input_size), interpolation = cv2.INTER_CUBIC))
#     return np.array(result).swapaxes(0,1).swapaxes(1,2)

# def get_img_coords(w, h, padding, crop_size):
#     new_w = w + 2*padding
#     new_h = h + 2*padding
#     cut_w = list(range(padding, new_w - padding, crop_size))
#     cut_h = list(range(padding, new_h - padding, crop_size))

#     list_hight = []
#     list_weight = []
#     for i in cut_h:
#         if i < new_h - padding - crop_size:
#             list_hight.append(i)
#     list_hight.append(new_h-crop_size-padding)

#     for i in cut_w:
#         if i < new_w - crop_size - padding:
#             list_weight.append(i)
#     list_weight.append(new_w-crop_size-padding)

#     img_coords = []
#     for i in list_weight:
#         for j in list_hight:
#             img_coords.append([i, j])
#     return img_coords

# def padded_for_org_img(values, num_band, padding):
#     padded_org_im = []
#     for i in range(num_band):
#         band = np.pad(values[i], padding, mode='reflect')
#         padded_org_im.append(band)

#     values = np.array(padded_org_im).swapaxes(0,1).swapaxes(1,2)
#     # print(values.shape)
#     del padded_org_im
#     return values

# def predict(model, values, img_coords, num_band, h, w, padding, crop_size, 
#             input_size, batch_size, thresh_hold, choose_stage):
#     cut_imgs = []
#     for i in range(len(img_coords)):
#         im = get_im_by_coord(values, img_coords[i][0], img_coords[i][1],
#                             num_band,padding, crop_size, input_size)
#         cut_imgs.append(im)

#     a = list(range(0, len(cut_imgs), batch_size))

#     if a[len(a)-1] != len(cut_imgs):
#         a[len(a)-1] = len(cut_imgs)

#     y_pred = []
#     for i in range(len(a)-1):
#         x_batch = []
#         x_batch = np.array(cut_imgs[a[i]:a[i+1]])
#         # print(x_batch.shape)
#         img_edge = []
#         # for img_x in x_batch:
#         #     lab_batch = color.rgb2lab(img_x)  
#             # img_edge.append(cv2.Canny(np.asarray(np.uint8(lab_batch)),0,0)[..., np.newaxis])
#         # print(img_edge.shape)
#         # img_edge = np.array(img_edge)
        
#         # print(x_batch.shape, img_edge.shape)
#         # y_batch = model.predict((x_batch/255, img_edge/255))
#         y_batch = model.predict(x_batch/255)
#         if len(model.outputs)>1:
#             y_batch = y_batch[choose_stage]
#         mutilabel = False
#         if y_batch.shape[-1]>=2:
#             mutilabel = True
#             y_batch = np.argmax(y_batch, axis=-1)
#         # print(np.unique(y_batch), y_batch.shape)
            
#         y_pred.extend(y_batch)
#     big_mask = np.zeros((h, w)).astype(np.float16)
#     for i in range(len(cut_imgs)):
#         true_mask = y_pred[i].reshape((input_size,input_size))
#         if not mutilabel:
#             true_mask = (true_mask>thresh_hold).astype(np.uint8)
#             true_mask = (cv2.resize(true_mask,(input_size, input_size), interpolation = cv2.INTER_CUBIC)>thresh_hold).astype(np.uint8)
#             # true_mask = true_mask.astype(np.float16)
#         start_x = img_coords[i][1]
#         start_y = img_coords[i][0]
#         big_mask[start_x-padding:start_x-padding+crop_size, start_y-padding:start_y -
#                     padding+crop_size] = true_mask[padding:padding+crop_size, padding:padding+crop_size]
#     del cut_imgs
#     return big_mask

# img_size = 256
# num_band = 3
# crop_size = 200
# batch_size = 1
# thresh_hold = 0.8
# choose_stage = 0

# model.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/mhanet/mhanet_change_road_256_1class_train.h5')
# image_path = '/home/quyet/DATA_ML/Projects/road_multi/crop/img/test.tif'
# image_path = glob.glob('/home/quyet/DATA_ML/Projects/change_detection/new_video/tarmac/img/*.tif')
# try: 
#     for i in tqdm(image_path):
#         if '_result.tif' not in i:
#             dataset = gdal.Open(i)
#             values = dataset.ReadAsArray()[0:num_band]
#             h,w = values.shape[1:3]    
#             padding = int((img_size - crop_size)/2)
#             img_coords = get_img_coords(w, h, padding, crop_size)
#             values = padded_for_org_img(values, num_band, padding)
#             big_mask = predict(model, values, img_coords, num_band, h, w, padding, crop_size, 
#                                 img_size, batch_size, thresh_hold, choose_stage)
#             big_mask[big_mask==0]=2
#             big_mask[big_mask==1]=0
#             big_mask[big_mask==2]=1
#             result_path = write_image(i, big_mask)
# except:
#     dataset = gdal.Open(image_path)
#     values = dataset.ReadAsArray()[0:num_band]
#     h,w = values.shape[1:3]    
#     padding = int((img_size - crop_size)/2)
#     img_coords = get_img_coords(w, h, padding, crop_size)
#     values = padded_for_org_img(values, num_band, padding)
#     big_mask = predict(model, values, img_coords, num_band, h, w, padding, crop_size, 
#                         img_size, batch_size, thresh_hold, choose_stage)
#     big_mask[big_mask==0]=2
#     big_mask[big_mask==1]=0
#     big_mask[big_mask==2]=1
#     result_path = write_image(image_path, big_mask)